In [1]:
from pathlib import Path 
import requests
#import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime 
from datetime import date,datetime
import pandas as pd



In [2]:

sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483*",
    security_token="1B6xNEvzIe5uroF6nRgKGTSk",
)

In [3]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [4]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(7287, 8)

In [5]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6991, 5)

In [6]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7293, 10)

fin empresas y comienzo de rescate de pacientes


In [7]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1753569, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [8]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1349255, 4)

In [9]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1349255, 13)

In [10]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [11]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [12]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [13]:
#pacientes_empresas22.shape

In [14]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [15]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [16]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [34]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Zurich"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [35]:
Pacientes_Archivo.head(10)

,poliza,Nombre Contratante,Per,Rut Titular,Dv,Nro Item,Relacion,nombres,Paterno,Materno,...,Vig Exclusion,Prima Base,Gneg,Desc Gneg,Subgrupo,Desc Subgrupo,Rut Cnt,email,Endoso Incl,Endoso Excl
0,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,7041483,K,0,T,VICENTE,CARCAMO,ALVAREZ,...,20230228,-0.0875,45,LABORATORIO CHILE,0,NaN,65195770,vicente.carcamo@gmail.com,3766,3810
1,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,7041483,K,3,C,MABEL,JARAMILLO,SILVA,...,20230228,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,vicente.carcamo@gmail.com,3766,3810
2,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8112405,1,0,T,GASTON,ABASOLO,GAROLERA,...,20230113,-0.0875,45,LABORATORIO CHILE,0,NaN,65195770,gaston.abasolo@laboratoriochile.cl,3766,3814
3,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8112405,1,1,C,LUISA NORA,BURGOS,VALDEBENITO,...,20230113,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,gaston.abasolo@laboratoriochile.cl,3766,3814
4,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8144624,5,5,H,JOSEFA CECILIA,LOPEZ,VERGARA,...,20220801,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,jose.lopez@laboratoriochile.cl,3766,3768
5,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8198710,6,3,H,DANIEL,SUAZO,OYARZUN,...,20220801,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,maria.oyarzun@laboratoriochile.cl,3766,3770
6,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8406341,K,2,H,MATIAS,FLORES,GLADE,...,20220801,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,GASTON.FLORES@LABORATORIOCHILE.CL,3766,3770
7,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,9121344,3,0,T,ALEJANDRO,SALTARINI,SANHUEZA,...,20221012,-0.0875,45,LABORATORIO CHILE,0,NaN,65195770,alejandro.saltarini@laboratoriochile.cl,3766,3836
8,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,9121344,3,1,H,FERNANDA,SALTARINI,CARRASCO,...,20221012,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,alejandro.saltarini@laboratoriochile.cl,3766,3836
9,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,9121344,3,2,H,RENATA,SALTARINI,CARRASCO,...,20221012,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,alejandro.saltarini@laboratoriochile.cl,3766,3836


In [36]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [37]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,poliza,Nombre Contratante,Per,Rut Titular,Dv,Nro Item,Relacion,nombres,Paterno,Materno,...,Prima Base,Gneg,Desc Gneg,Subgrupo,Desc Subgrupo,Rut Cnt,email,Endoso Incl,Endoso Excl,rut
0,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,7041483,K,0,T,VICENTE,CARCAMO,ALVAREZ,...,-0.0875,45,LABORATORIO CHILE,0,NaN,65195770,vicente.carcamo@gmail.com,3766,3810,7041483K
1,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,7041483,K,3,C,MABEL,JARAMILLO,SILVA,...,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,vicente.carcamo@gmail.com,3766,3810,75113870
2,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8112405,1,0,T,GASTON,ABASOLO,GAROLERA,...,-0.0875,45,LABORATORIO CHILE,0,NaN,65195770,gaston.abasolo@laboratoriochile.cl,3766,3814,81124051
3,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8112405,1,1,C,LUISA NORA,BURGOS,VALDEBENITO,...,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,gaston.abasolo@laboratoriochile.cl,3766,3814,7429295K
4,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8144624,5,5,H,JOSEFA CECILIA,LOPEZ,VERGARA,...,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,jose.lopez@laboratoriochile.cl,3766,3768,187985080


In [38]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_26156\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,poliza,Nombre Contratante,Per,Rut Titular,Dv,Nro Item,Relacion,nombres,Paterno,Materno,...,Prima Base,Gneg,Desc Gneg,Subgrupo,Desc Subgrupo,Rut Cnt,email,Endoso Incl,Endoso Excl,rut
0,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,7041483,K,0,T,VICENTE,CARCAMO,ALVAREZ,...,-0.0875,45,LABORATORIO CHILE,0,NaN,65195770,vicente.carcamo@gmail.com,3766,3810,7041483K
1,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,7041483,K,3,C,MABEL,JARAMILLO,SILVA,...,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,vicente.carcamo@gmail.com,3766,3810,75113870
2,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8112405,1,0,T,GASTON,ABASOLO,GAROLERA,...,-0.0875,45,LABORATORIO CHILE,0,NaN,65195770,gaston.abasolo@laboratoriochile.cl,3766,3814,81124051
3,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8112405,1,1,C,LUISA NORA,BURGOS,VALDEBENITO,...,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,gaston.abasolo@laboratoriochile.cl,3766,3814,7429295K
4,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8144624,5,5,H,JOSEFA CECILIA,LOPEZ,VERGARA,...,0.0000,45,LABORATORIO CHILE,0,NaN,65195770,jose.lopez@laboratoriochile.cl,3766,3768,187985080


In [39]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_26156\192164855.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_26156\192164855.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [40]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(4109, 39)

In [41]:
Pacientes_con_id.head()

,poliza,Nombre Contratante,Per,Rut Titular,Dv,Nro Item,Relacion,nombres,Paterno,Materno,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,7041483,K,0,T,Vicente,CARCAMO,ALVAREZ,...,True,0018c00002ei7H5AAI,7041483K,Vicente Carcamo Alvarez,RUT,None,None,None,Hombre,None
1,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,7041483,K,3,C,Mabel,JARAMILLO,SILVA,...,True,0018c00002ei7H6AAI,75113870,Mabel Jaramillo Silva,RUT,None,None,None,Mujer,None
2,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8112405,1,0,T,Gaston,ABASOLO,GAROLERA,...,True,0018c00002ei7HFAAY,81124051,Gaston Abasolo Garolera,RUT,None,None,None,Hombre,None
3,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8112405,1,1,C,Luisa Nora,BURGOS,VALDEBENITO,...,True,0018c00002ei7HGAAY,7429295K,Luisa Nora Burgos Valdebenito,RUT,None,None,None,Mujer,None
4,121292,CORP.DE BIENESTAR TRABAJADORES LABORATORIO CHILE,25,8144624,5,5,H,Josefa Cecilia,LOPEZ,VERGARA,...,True,0018c00002mTw0VAAS,187985080,Josefa Cecilia Lopez Vergara,RUT,None,None,None,Mujer,None


In [42]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("Nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [43]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
fecha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)



Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
#clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)


In [44]:
print(dia_hoy)

12


In [45]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
159067,a1Y8c00000C6VurEAF,0018c00002amTLEAA2,a1W8c0000083oyCEAQ,Zurich,0018c00002f2ufSAAQ,Antofagasta Minerals S.A.,Antofagasta Minerals S.A.,Zurich,Antofagasta Minerals S.A.,313033,None,EMP9018,A-8049
159080,a1Y8c00000C6VvGEAV,0018c00002amLUiAAM,a1W8c0000083ouBEAQ,Zurich,0018c00002f2pM1AAI,Antofagasta Minerals S.A.,Antofagasta Minerals S.A.,Zurich,Antofagasta Minerals S.A.,313009,None,EMP9016,A-8047
159091,a1Y8c00000C6VvlEAF,0018c00002amLbvAAE,a1W8c0000083ouBEAQ,Zurich,0018c00002f2pM1AAI,Antofagasta Minerals S.A.,Antofagasta Minerals S.A.,Zurich,Antofagasta Minerals S.A.,313009,None,EMP9016,A-8047
159253,a1Y8c00000C6WGrEAN,0018c00002eTelgAAC,a1W8c000007vDVmEAM,Zurich,0018c00002axQh4AAE,Akzio Consultores Ltda.,Akzio Consultores Ltda.,Zurich,Akzio Consultores Ltda.,364427,None,EMP6129,A-0536
159254,a1Y8c00000C6WGsEAN,0018c00002eTelhAAC,a1W8c000007vDVmEAM,Zurich,0018c00002axQh4AAE,Akzio Consultores Ltda.,Akzio Consultores Ltda.,Zurich,Akzio Consultores Ltda.,364427,None,EMP6129,A-0536


In [46]:
campaña_seleccionada.shape

(39150, 13)

In [47]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [ ]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [ ]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)


In [ ]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)